In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
program = "graph_coloring"
# program = "maximal_matching"
# program = "dijkstra_token_ring"
graph_name = "graph_7"

no_simulations = 100000
fault_interval = 1
sched = 0

selected_nodes = [0, 4, 9]

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__duong"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,0,6419.256840
1,1,6024.680462
2,2,5654.264227
3,3,5306.528797
4,4,4980.085413
...,...,...
206,206,-24.834306
207,207,-24.835183
208,208,-24.836007
209,209,-24.836781


In [5]:
dfs[0]

,Steps,exp_fit
0,0,4216.179227
1,1,4023.876853
2,2,3840.386310
3,3,3665.303816
4,4,3498.244091
...,...,...
426,426,19.525723
427,427,19.525723
428,428,19.525722
429,429,19.525722


In [6]:
dfs[1]

,Steps,exp_fit
0,0,5665.162904
1,1,5355.882692
2,2,5063.405212
3,3,4786.817595
4,4,4525.256569
...,...,...
168,168,-27.138748
169,169,-27.164733
170,170,-27.189306
171,171,-27.212544


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__4,exp_fit__9
Steps,,,
0,4216.179227,5665.162904,6419.256840
1,4023.876853,5355.882692,6024.680462
2,3840.386310,5063.405212,5654.264227
3,3665.303816,4786.817595,5306.528797
4,3498.244091,4525.256569,4980.085413
...,...,...,...
168,21.112732,-27.138748,-24.690490
169,21.040010,-27.164733,-24.700173
170,20.970621,-27.189306,-24.709264


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 4,Exponential fit of node 9
Steps,,,
0,4216.179227,5665.162904,6419.256840
1,4023.876853,5355.882692,6024.680462
2,3840.386310,5063.405212,5654.264227
3,3665.303816,4786.817595,5306.528797
4,3498.244091,4525.256569,4980.085413
...,...,...,...
168,21.112732,-27.138748,-24.690490
169,21.040010,-27.164733,-24.700173
170,20.970621,-27.189306,-24.709264


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}__{fault_interval}.png",
    bbox_inches="tight",
)
plt.close()